In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:1"

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 32, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
        
        self.lineartest0 = nn.Linear(2, emsize)
        self.lineartest1 = nn.Linear(2, emsize)
        self.lineartest2 = nn.Linear(2, emsize)
        self.lineartest3 = nn.Linear(2 * emsize, emsize)
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 2]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def test_encode(self, src): #src must be [batchsize * nsamples, npoints = 5, 2]
        point_1 = self.lineartest1(src[:, 3, :] - 0.5) #want mean 0!!!!
        point_2 = self.lineartest1(src[:, 4, :] - 0.5)
        remaining = self.lineartest0(src[:, :3, :]) #[batchsize * nsamples, 3, emsize]
        point_1_message = self.lineartest1(src[:, 3, :] - 0.5).squeeze(1)
        point_2_message = self.lineartest2(src[:, 4, :] - 0.5).squeeze(1)
        point_1 = F.relu(torch.cat([point_1, point_2_message], dim = 1))
        point_2 = F.relu(torch.cat([point_2, point_1_message], dim = 1))
        point_1 = self.lineartest3(point_1)
        point_2 = self.lineartest3(point_2)
        src = torch.cat([torch.zeros_like(remaining.transpose(0, 1)), point_1.unsqueeze(0), point_2.unsqueeze(0)])
        return src #[npoints = 5, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 3], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 3, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 3, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 3, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, dtype = torch.bool, device = device).triu(1) #[npoints - 3, npoints - 3], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters(), lr = 0.0001)


class environment():
    def reset(self, npoints, batchsize, nsamples):
        if npoints <= 3:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = batchsize * nsamples #so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = torch.rand([batchsize, npoints - 3, 2], device = device).unsqueeze(1).expand(-1, nsamples, -1, -1).reshape(self.batchsize, npoints - 3, 2)
        self.corner_points = torch.tensor([[0, 0], [2, 0], [0, 2]], dtype = torch.float, device = device)
        self.points = torch.cat([self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1), self.points], dim = -2) #[batchsize * nsamples, npoints, 2]
        self.points_mask = torch.cat([torch.ones([self.batchsize, 3], dtype = torch.bool, device = device), torch.zeros([self.batchsize, npoints - 3], dtype = torch.bool, device = device)], dim = 1)
        self.points_sequence = torch.empty([self.batchsize, 0], dtype = torch.long, device = device)
        
        """use a trick, for the purpose of an 'external' triangle that is always left untouched, which means we don't have to deal with boundary edges as being different. external triangle is [0, 1, 2] traversed clockwise..."""
        self.partial_delaunay_triangles = torch.tensor([[0, 2, 1], [0, 1, 2]], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1, -1).contiguous() #[batchsize, ntriangles, 3] contains index of points, always anticlockwise
        self.partial_delaunay_edges = torch.tensor([5, 4, 3, 2, 1, 0], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1).contiguous() #[batchsize, ntriangles * 3] contains location of corresponding edge (edges go in order 01, 12, 20). Edges will always flip since triangles are stored anticlockwise.
        
        self.ntriangles = 2 #can store as scalar, since will always be the same
        self.cost = torch.zeros([self.batchsize], device = device)
        
        self.logprob = torch.zeros([self.batchsize], device = device, requires_grad = True)
    
    def update(self, point_index): #point_index is [batchsize]
        if point_index.size(0) != self.batchsize:
            print("Error: point_index.size() doesn't match expected size, should be [batchsize]")
            return
        if self.points_mask.gather(1, point_index.unsqueeze(1)).sum():
            print("Error: some points already added")
            return
        triangles_coordinates = self.points.gather(1, self.partial_delaunay_triangles.view(self.batchsize, self.ntriangles * 3).unsqueeze(2).expand(-1, -1, 2)).view(self.batchsize, self.ntriangles, 3, 2) #[batchsize, ntriangles, 3, 2]
        newpoint = self.points.gather(1, point_index.unsqueeze(1).unsqueeze(2).expand(self.batchsize, 1, 2)).squeeze(1) #[batchsize, 2]
        
        incircle_matrix = torch.cat([triangles_coordinates, newpoint.unsqueeze(1).unsqueeze(2).expand(-1, self.ntriangles, 1, -1)], dim = -2) #[batchsize, ntriangles, 4, 2]
        incircle_matrix = torch.cat([incircle_matrix, (incircle_matrix * incircle_matrix).sum(-1, keepdim = True), torch.ones([self.batchsize, self.ntriangles, 4, 1], device = device)], dim = -1) #[batchsize, ntriangles, 4, 4]
        incircle_test = incircle_matrix.det() > 0 #[batchsize, ntriangles], is True if inside incircle
        removed_edge_mask = incircle_test.unsqueeze(2).expand(-1, -1, 3).reshape(-1) #[batchsize * ntriangles * 3]
        
        edges = (self.partial_delaunay_edges + self.ntriangles * 3 * torch.arange(self.batchsize, device = device).unsqueeze(1)).view(-1) #[batchsize * ntriangles * 3]
        neighbouring_edge = edges.masked_select(removed_edge_mask)
        neighbouring_edge_mask = torch.zeros([self.batchsize * self.ntriangles * 3], device = device, dtype = torch.bool)
        neighbouring_edge_mask[neighbouring_edge] = True
        neighbouring_edge_mask = (neighbouring_edge_mask * removed_edge_mask.logical_not()) #[batchsize * ntriangles * 3]
        
        n_new_triangles = neighbouring_edge_mask.view(self.batchsize, -1).sum(-1) #[batchsize]
        
        new_point = point_index.unsqueeze(1).expand(-1, self.ntriangles * 3).masked_select(neighbouring_edge_mask.view(self.batchsize, -1))
        
        second_point_mask = neighbouring_edge_mask.view(self.batchsize, -1, 3) #[batchsize, ntriangles 3]
        (first_point_indices0, first_point_indices1, first_point_indices2) = second_point_mask.nonzero(as_tuple = True)
        first_point_indices2 = (first_point_indices2 != 2) * (first_point_indices2 + 1)
        
        first_point = self.partial_delaunay_triangles[first_point_indices0, first_point_indices1, first_point_indices2] #[?]
        second_point = self.partial_delaunay_triangles.masked_select(second_point_mask) #[?]
        
        new_triangles_mask = torch.cat([incircle_test, torch.ones([self.batchsize, 2], dtype = torch.bool, device = device)], dim = 1) #[batchsize, ntriangles + 2]
        
        new_neighbouring_edges = 3 * new_triangles_mask.nonzero(as_tuple = True)[1] #[?], 3* since is the 01 edge of new triangles (see later)
        self.partial_delaunay_edges.masked_scatter_(neighbouring_edge_mask.view(self.batchsize, -1), new_neighbouring_edges) #still [batchsize, ntriangles * 3] for now
        
        self.partial_delaunay_triangles = torch.cat([self.partial_delaunay_triangles, torch.empty([self.batchsize, 2, 3], dtype = torch.long, device = device)], dim = 1)
        self.partial_delaunay_edges = torch.cat([self.partial_delaunay_edges, torch.empty([self.batchsize, 6], dtype = torch.long, device = device)], dim = 1)
        new_triangles = torch.stack([first_point, second_point, new_point], dim = 1) #[?, 3], edge here is flipped compared to edge in neighbouring triangle (so first_point is the second point in neighbouring edge)
        self.partial_delaunay_triangles.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3), new_triangles) #[batchsize, ntriangles + 2, 3]
        
        new_edge01 = neighbouring_edge_mask.view(self.batchsize, -1).nonzero(as_tuple = True)[1] #[?]
        
        """we are currently storing which triangles have to be inserted, via the edges along the perimeter of the delaunay cavity, we need to compute which edge is to the 'left'/'right' of each edge"""
        """don't have the memory to do a batchsize * n * n boolean search, don't have the speed to do a batchsize^2 search (as would be the case for sparse matrix or similar)"""
        """best alternative: rotate the edge around right point, repeat until hit edge in mask (will never go to an edge of a removed triangle before we hit edge in mask) should basically be order 1!!!!!"""
        
        neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?]
        next_neighbouring_edge_index = torch.empty_like(neighbouring_edge_index) #[?]
        
        rotating_flipped_neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?], initialise
        todo_mask = torch.ones_like(next_neighbouring_edge_index, dtype = torch.bool) #[?]
        while todo_mask.sum():
            rotating_neighbouring_edge_index = rotating_flipped_neighbouring_edge_index + 1 - 3 * (rotating_flipped_neighbouring_edge_index % 3 == 2) #[todo_mask.sum()], gets smaller until nothing left EFFICIENCY (this may be seriously stupid, as it requires making a bunch of copies when I could be doing stuff inplace)
            
            update_mask = neighbouring_edge_mask[rotating_neighbouring_edge_index] #[todo_mask.sum()]
            update_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(todo_mask, update_mask) #[?]
            
            next_neighbouring_edge_index.masked_scatter_(update_mask_unravel, rotating_neighbouring_edge_index.masked_select(update_mask)) #[?]
            
            todo_mask.masked_fill_(update_mask_unravel, False) #[?]
            rotating_flipped_neighbouring_edge_index = edges[rotating_neighbouring_edge_index.masked_select(update_mask.logical_not())] #[todo_mask.sum()]
        triangle_index = new_triangles_mask.view(-1).nonzero(as_tuple = True)[0] #[?], index goes up to batchsize * (ntriangles + 2), this is needed for when we invert the permutation by scattering (won't scatter same number of triangles per batch)
        
        next_triangle_index = torch.empty_like(edges).masked_scatter_(neighbouring_edge_mask, triangle_index)[next_neighbouring_edge_index] #[?], index goes up to batchsize * (ntriangles + 2)
        next_edge = 3 * next_triangle_index + 1 #[?]
        
        invert_permutation = torch.empty_like(new_triangles_mask.view(-1), dtype=torch.long) #[batchsize * (ntriangles + 2)]
        invert_permutation[next_triangle_index] = triangle_index #[batchsize * (ntriangles + 2)]
        previous_triangle_index = invert_permutation.masked_select(new_triangles_mask.view(-1)) #[?]
        previous_edge = 3 * previous_triangle_index + 2 #[?]
        
        """in the above we rotated around 'first_point' in our new triangles"""
        new_edge20 = next_edge % ((self.ntriangles + 2) * 3) #[?]
        new_edge12 = previous_edge % ((self.ntriangles + 2) * 3) #[?]
        
        new_edges = torch.stack([new_edge01, new_edge12, new_edge20], dim = 1) #[?, 3]
        self.partial_delaunay_edges.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3).reshape(self.batchsize, -1), new_edges) #[batchsize, (ntriangles + 2) * 3]
        
        self.ntriangles += 2
        """currently only count the extra triangles you replace (not the on you have to remove because you're located there, and not the ones you make because you have to create two more"""
        self.cost += (n_new_triangles - 3)
        self.points_mask.scatter_(1, point_index.unsqueeze(1).expand(-1, self.npoints), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 300000, npoints = 13, batchsize = 2000, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        NN.eval()
        with torch.no_grad():
            for j in range(3, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        regular_reinforcement = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1)
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 2
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                #* (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 25) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss with thresholding
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * regular_reinforcement * (positive_reinforcement_binary + negative_reinforcement_binary * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 40))).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item())#, env.logprob[0].item())
  

In [3]:
train(npoints = 53, batchsize = 800, nsamples = 8)

95.70874786376953 -2.7189157009124756 -5.273835182189941 -2.7654266357421875 -2.1188812255859375
95.37625122070312 -2.7622036933898926 -5.337827682495117 -2.95751953125 -3.47662353515625
95.62749481201172 -2.7169365882873535 -5.252511978149414 -1.9066963195800781 -6.614990234375
95.45999908447266 -2.714162588119507 -5.257511138916016 -4.0639495849609375 -6.5550384521484375
95.21499633789062 -2.708631753921509 -5.201584815979004 -3.2781524658203125 -6.642524719238281
95.51749420166016 -2.7353463172912598 -5.327310085296631 -2.3954696655273438 -6.202178955078125
95.18374633789062 -2.694822311401367 -5.242446422576904 -2.4687957763671875 -6.86376953125
94.8949966430664 -2.751927375793457 -5.346766471862793 -2.4630126953125 -7.789588928222656
95.55500030517578 -2.769659996032715 -5.3223490715026855 -1.95404052734375 -5.21453857421875
95.72374725341797 -2.806429147720337 -5.366363525390625 -1.85101318359375 -6.254737854003906
95.3062515258789 -2.7985119819641113 -5.376979351043701 -1.806427

94.62124633789062 -2.439905881881714 -4.746460914611816 -3.1589622497558594 -3.1389591693878174
94.39249420166016 -2.462801456451416 -4.770102024078369 -1.202444076538086 -4.669429779052734
94.01374816894531 -2.447542905807495 -4.720329761505127 -1.759016990661621 -4.120563507080078
94.47249603271484 -2.4582247734069824 -4.769865036010742 -1.363576889038086 -6.425626754760742
94.73999786376953 -2.504643440246582 -4.871433258056641 -2.2384719848632812 -9.317033767700195
94.84749603271484 -2.502892017364502 -4.854456424713135 -2.4727253913879395 -1.2870712280273438
94.90249633789062 -2.4842870235443115 -4.74656867980957 -0.8819141387939453 -5.5459136962890625
94.77124786376953 -2.5208613872528076 -4.9049224853515625 -2.0636978149414062 -4.0251007080078125
94.72750091552734 -2.5526604652404785 -4.8746867179870605 -1.9112472534179688 -5.91558837890625
94.0 -2.4471957683563232 -4.747005462646484 -2.308624267578125 -3.1275367736816406
94.3375015258789 -2.495455503463745 -4.823144435882568 -1

94.62124633789062 -2.4250948429107666 -4.7621750831604 -1.8214197158813477 -3.4004287719726562
94.47624969482422 -2.4300553798675537 -4.799479961395264 -1.9099550247192383 -3.178269863128662
94.89875030517578 -2.4279189109802246 -4.759355068206787 -2.1528968811035156 -7.14885139465332
95.02124786376953 -2.4557766914367676 -4.813920974731445 -3.895435333251953 -1.32951021194458
94.86249542236328 -2.4841127395629883 -4.856939315795898 -1.750382423400879 -4.408498764038086
94.65999603271484 -2.423814296722412 -4.729470729827881 -1.8211174011230469 -1.0620079040527344
94.4949951171875 -2.480642557144165 -4.850656509399414 -1.3907108306884766 -2.8541641235351562
95.04999542236328 -2.429168224334717 -4.822264671325684 -4.534424781799316 -10.891925811767578
94.80999755859375 -2.4644010066986084 -4.78567361831665 -5.803871154785156 -7.255657196044922
94.94249725341797 -2.4917941093444824 -4.83619499206543 -1.368072509765625 -1.6147584915161133
94.62374877929688 -2.484229803085327 -4.7994594573

94.00125122070312 -2.7950692176818848 -5.362489700317383 -2.6414318084716797 -1.1196889877319336
94.2449951171875 -2.82597279548645 -5.534890651702881 -3.0356650352478027 -3.759383201599121
93.85250091552734 -2.819606065750122 -5.497504234313965 -3.986631393432617 -2.599719524383545
94.10749816894531 -2.799302101135254 -5.479548454284668 -3.180330276489258 -6.244041442871094
94.2699966430664 -2.86612606048584 -5.578289031982422 -2.8534488677978516 -2.733156204223633
94.29124450683594 -2.9379522800445557 -5.686798095703125 -4.146381378173828 -7.020416259765625
93.90249633789062 -2.8862903118133545 -5.6715989112854 -4.753361225128174 -1.572244644165039
94.14749908447266 -2.930137872695923 -5.649896144866943 -0.845008134841919 -5.688391208648682
93.82125091552734 -2.9056549072265625 -5.666751384735107 -3.339144706726074 -3.751223564147949
94.13249969482422 -3.0106725692749023 -5.773723602294922 -2.9541282653808594 -4.228694915771484
94.10749816894531 -2.9989542961120605 -5.804587841033935

93.75749969482422 -2.864149808883667 -5.637737274169922 -3.847993850708008 -3.5945987701416016
93.98500061035156 -2.938854694366455 -5.679155349731445 -2.9215126037597656 -3.9797792434692383
93.78250122070312 -2.8525524139404297 -5.599421501159668 -3.776355743408203 -7.195289611816406
93.87249755859375 -2.919750213623047 -5.594842433929443 -2.909210205078125 -5.131988525390625
93.72874450683594 -2.8946471214294434 -5.647032737731934 -2.711851119995117 -5.502187728881836
94.01749420166016 -2.948134660720825 -5.767446041107178 -2.5560598373413086 -5.531616687774658
93.97624969482422 -2.944347381591797 -5.773106575012207 -2.5700135231018066 -6.018260955810547
93.67499542236328 -2.9430973529815674 -5.791335582733154 -2.0555567741394043 -8.079171180725098
94.07499694824219 -3.0166518688201904 -5.778085708618164 -2.103586196899414 -13.003081321716309
93.54249572753906 -3.0405914783477783 -5.859330177307129 -2.5602431297302246 -1.7364788055419922
93.65499877929688 -2.976571559906006 -5.765049

...

91.85124969482422 -4.475749492645264 -8.27188491821289 -4.44622802734375 -15.585060119628906
91.9012451171875 -4.450135707855225 -8.355367660522461 -4.659526824951172 -7.490478515625
91.78874969482422 -4.367873668670654 -8.120210647583008 -4.010009765625 -5.937950134277344
91.54499816894531 -4.37390661239624 -8.110363960266113 -3.7310791015625 -3.3942763805389404
91.56124877929688 -4.2410736083984375 -7.950778484344482 -4.39825439453125 -2.9677886962890625
91.54000091552734 -4.149557113647461 -7.766199588775635 -3.97821044921875 -2.0906982421875
91.66500091552734 -4.1974968910217285 -7.8405537605285645 -4.89215087890625 -14.40264892578125
91.25374603271484 -4.046245098114014 -7.54128360748291 -4.08612060546875 -12.311019897460938
91.51874542236328 -4.125508785247803 -7.598188877105713 -6.318572998046875 -8.04153823852539
91.5262451171875 -4.031107425689697 -7.553972244262695 -5.0365142822265625 -6.7570343017578125
90.97999572753906 -4.003755569458008 -7.4636454582214355 -4.215637207031

93.28374481201172 -5.854883193969727 -10.493562698364258 -8.243072509765625 -6.789148330688477
93.16374969482422 -5.84251070022583 -10.450870513916016 -6.594757080078125 -10.650772094726562
93.20249938964844 -5.8388824462890625 -10.495720863342285 -4.852325439453125 -10.368560791015625
92.92874908447266 -5.764017105102539 -10.393118858337402 -4.701019287109375 -9.657012939453125
93.2762451171875 -5.689849853515625 -10.334892272949219 -7.9217529296875 -15.911575317382812
93.55999755859375 -5.914945602416992 -10.690299987792969 -5.0015106201171875 -9.7147216796875
93.95499420166016 -5.941414833068848 -10.720979690551758 -5.9826507568359375 -10.0059814453125
94.06749725341797 -6.16322660446167 -11.08445930480957 -11.942672729492188 -13.596580505371094
93.82374572753906 -6.27437686920166 -11.272393226623535 -4.122711181640625 -11.883758544921875
94.07749938964844 -6.160182952880859 -11.15511703491211 -8.832015991210938 -15.072052001953125
94.66999816894531 -6.231340408325195 -11.3749103546

92.23624420166016 -4.22859001159668 -7.95234489440918 -3.5291061401367188 -10.264793395996094
92.23249816894531 -4.195612907409668 -7.909787178039551 -4.8038177490234375 -4.9796295166015625
92.3949966430664 -4.193716526031494 -7.796070098876953 -5.278167724609375 -9.407119750976562
92.59749603271484 -4.249004364013672 -7.912264347076416 -3.32867431640625 -13.102645874023438
92.19249725341797 -4.180573463439941 -7.796498775482178 -3.9711456298828125 -5.1584625244140625
92.5562515258789 -4.226674556732178 -7.922501087188721 -4.968910217285156 -7.984954833984375
92.66374969482422 -4.406539440155029 -8.296133041381836 -3.681427001953125 -10.212692260742188
92.51749420166016 -4.325344085693359 -8.109235763549805 -5.1063995361328125 -11.578277587890625
92.8074951171875 -4.483373165130615 -8.274495124816895 -3.8731842041015625 -6.839508056640625
92.69499969482422 -4.486715793609619 -8.330585479736328 -4.41876220703125 -6.7381744384765625
92.80874633789062 -4.641049861907959 -8.560722351074219

90.76124572753906 -3.8468453884124756 -7.33013391494751 -3.7432861328125 -6.7035980224609375
90.70375061035156 -3.864255666732788 -7.3233842849731445 -4.084205627441406 -7.967498779296875
90.34874725341797 -3.7822887897491455 -7.101604461669922 -2.75604248046875 -6.6502685546875
90.7762451171875 -3.7846624851226807 -7.064952373504639 -2.661102294921875 -3.8898391723632812
90.57875061035156 -3.7332959175109863 -7.038899898529053 -2.731719970703125 -9.067672729492188
90.50249481201172 -3.6159679889678955 -6.897128105163574 -3.4997406005859375 -6.7818756103515625
90.28624725341797 -3.6915135383605957 -7.00545072555542 -2.5566749572753906 -7.3834075927734375
90.29249572753906 -3.608206033706665 -6.876624584197998 -3.1332855224609375 -6.704437255859375
90.34124755859375 -3.5234336853027344 -6.660308361053467 -3.3382720947265625 -6.558815002441406
90.31499481201172 -3.5272293090820312 -6.627852916717529 -4.844028472900391 -7.81024169921875
90.22624969482422 -3.5269839763641357 -6.71009349822

KeyboardInterrupt: 

In [2]:
NN.load_state_dict(torch.load('2D_50points_small_88'))

<All keys matched successfully>

In [ ]:
train(npoints = 53, batchsize = 800, nsamples = 8)

88.18875122070312 -2.7180540561676025 -5.248782157897949 -1.913238525390625 -7.618043899536133
88.02249908447266 -2.778900623321533 -5.318701267242432 -3.3138427734375 -3.14471435546875
87.85749816894531 -2.7197744846343994 -5.318726062774658 -5.218583106994629 -6.78717041015625
88.17499542236328 -2.7519891262054443 -5.23708963394165 -1.71710205078125 -2.9439449310302734
88.11249542236328 -2.7357373237609863 -5.330355167388916 -1.7489776611328125 -3.7662200927734375
87.87749481201172 -2.7110867500305176 -5.236109733581543 -1.1627960205078125 -8.691793441772461
88.69999694824219 -2.761061906814575 -5.362802982330322 -2.4369068145751953 -8.948326110839844
88.01499938964844 -2.747528553009033 -5.2635579109191895 -1.5021209716796875 -8.090530395507812
88.41374969482422 -2.7736458778381348 -5.340282440185547 -2.769317626953125 -6.3187408447265625
88.09500122070312 -2.7061967849731445 -5.303525924682617 -2.752105712890625 -6.07769775390625
88.23749542236328 -2.7461559772491455 -5.26331281661

88.70249938964844 -2.901174783706665 -5.6060099601745605 -2.2674560546875 -3.270406723022461
88.36624908447266 -2.8766512870788574 -5.556922435760498 -3.614748001098633 -1.3513946533203125
88.32249450683594 -2.96258282661438 -5.647995948791504 -2.278045654296875 -3.9591217041015625
88.24624633789062 -3.0044472217559814 -5.754491329193115 -2.1424360275268555 -3.4862823486328125
88.14374542236328 -2.970349073410034 -5.652562141418457 -2.9597015380859375 -3.05950927734375
88.34124755859375 -2.959721088409424 -5.636378288269043 -1.4139404296875 -11.904571533203125
88.58499908447266 -2.9969780445098877 -5.796715259552002 -2.5954208374023438 -11.377961158752441
88.10749816894531 -2.9053075313568115 -5.581669807434082 -1.925872564315796 -2.990875244140625
88.38999938964844 -2.9563896656036377 -5.60609245300293 -1.741851806640625 -7.81536865234375
88.13874816894531 -3.0210788249969482 -5.783102035522461 -3.266265869140625 -8.877288818359375
88.45999908447266 -2.9839701652526855 -5.641681671142

88.55999755859375 -3.152808666229248 -5.979778289794922 -4.873422622680664 -7.049407958984375
88.5875015258789 -3.0973613262176514 -5.924673557281494 -3.189361572265625 -3.673532485961914
88.49874877929688 -3.1448140144348145 -5.981250762939453 -1.5887298583984375 -1.4053955078125
89.06124877929688 -3.222752094268799 -6.04863977432251 -3.036834716796875 -8.466995239257812
88.91874694824219 -3.111501455307007 -5.925888538360596 -1.3899259567260742 -5.872039794921875
88.31375122070312 -3.050903797149658 -5.824767589569092 -4.179684638977051 -7.7418060302734375
88.50499725341797 -3.0850861072540283 -5.820024013519287 -3.6859073638916016 -5.477886199951172
88.7074966430664 -3.0834012031555176 -5.905383586883545 -3.0442886352539062 -6.266357421875
88.73999786376953 -3.0890862941741943 -5.9093852043151855 -3.96954345703125 -3.2240381240844727
88.17874908447266 -3.0444445610046387 -5.8496174812316895 -3.3971710205078125 -5.5904388427734375
88.75499725341797 -3.115586757659912 -5.9242444038391

88.44124603271484 -3.267754554748535 -6.179652690887451 -2.4635772705078125 -5.390253067016602
88.6812515258789 -3.32796573638916 -6.325140953063965 -2.8721656799316406 -10.991439819335938
88.07624816894531 -3.254415273666382 -6.1751322746276855 -2.9317626953125 -4.4730224609375
88.84874725341797 -3.316920757293701 -6.212652206420898 -4.039344787597656 -18.637910842895508
88.20874786376953 -3.4194390773773193 -6.371285438537598 -5.889167785644531 -7.2408294677734375
88.17375183105469 -3.2706799507141113 -6.201554298400879 -4.794647216796875 -3.27593994140625
88.74624633789062 -3.3278379440307617 -6.2861175537109375 -1.305145263671875 -2.643160820007324
88.29624938964844 -3.3817543983459473 -6.3343119621276855 -1.7404251098632812 -2.2236175537109375
88.4637451171875 -3.3462207317352295 -6.332103729248047 -1.6128273010253906 -5.045648574829102
88.62249755859375 -3.3465752601623535 -6.361260414123535 -1.5147514343261719 -9.259490966796875
88.56375122070312 -3.387612819671631 -6.4025168418

88.36125183105469 -3.1373393535614014 -5.977631092071533 -3.5241851806640625 -11.787673950195312
88.25125122070312 -3.1055908203125 -5.935276031494141 -2.0883445739746094 -9.68389892578125
88.2562484741211 -3.072783946990967 -5.799047946929932 -4.12628173828125 -7.398717880249023
88.16500091552734 -3.0884816646575928 -5.8481221199035645 -4.579567909240723 -7.3055419921875
88.00749969482422 -3.065782070159912 -5.773090362548828 -3.8077545166015625 -2.697479248046875
88.45124816894531 -3.0832183361053467 -5.819550037384033 -6.458099365234375 -4.634258270263672
88.5625 -3.0162158012390137 -5.761972427368164 -2.6949920654296875 -7.998260498046875
88.50999450683594 -3.1094436645507812 -5.937357425689697 -3.008026123046875 -6.0689697265625
88.62749481201172 -3.045830726623535 -5.8434062004089355 -1.0971221923828125 -7.6512298583984375
88.31375122070312 -3.1375176906585693 -5.9576897621154785 -3.8014602661132812 -8.997047424316406
88.55500030517578 -3.1006274223327637 -5.916897296905518 -2.52

...

88.20249938964844 -2.8534977436065674 -5.426479339599609 -1.64501953125 -2.822296142578125
88.29249572753906 -2.8028628826141357 -5.423936367034912 -3.275909423828125 -4.102264404296875
88.19249725341797 -2.8040173053741455 -5.3740386962890625 -5.121173858642578 -4.0621795654296875
87.72124481201172 -2.809932231903076 -5.432948589324951 -2.173065185546875 -1.8419952392578125
88.05874633789062 -2.7587547302246094 -5.333837032318115 -2.669008255004883 -2.840301513671875
87.9324951171875 -2.785780668258667 -5.376680850982666 -2.710968017578125 -8.379627227783203
88.2387466430664 -2.7530784606933594 -5.360018253326416 -1.3208351135253906 -3.707895278930664
88.23999786376953 -2.728128671646118 -5.265081405639648 -3.248809814453125 -3.080371856689453
88.41374969482422 -2.756681442260742 -5.289231300354004 -1.3164215087890625 -5.522699356079102
88.04374694824219 -2.808690309524536 -5.344399929046631 -2.2925453186035156 -3.3972320556640625
88.33499908447266 -2.8229401111602783 -5.3951115608215

88.32249450683594 -2.698834180831909 -5.148734092712402 -2.04852294921875 -2.3726654052734375
88.41749572753906 -2.7210278511047363 -5.266143798828125 -1.32843017578125 -4.1239013671875
87.81499481201172 -2.688228130340576 -5.225466251373291 -1.36273193359375 -7.387392044067383
88.25749969482422 -2.7144267559051514 -5.245112419128418 -2.283416748046875 -3.942657470703125
88.01499938964844 -2.6758217811584473 -5.156291961669922 -3.368560791015625 -9.199249267578125
88.41124725341797 -2.655083179473877 -5.100444316864014 -5.382366180419922 -3.8551864624023438
88.34124755859375 -2.7222647666931152 -5.2542033195495605 -2.6584701538085938 -3.9239654541015625
88.09749603271484 -2.6955296993255615 -5.174220561981201 -3.4561614990234375 -8.04547119140625
88.02249908447266 -2.661713123321533 -5.120666980743408 -3.249195098876953 -8.90522575378418
88.48625183105469 -2.640026807785034 -5.133514881134033 -2.362030029296875 -4.01055908203125
88.45124816894531 -2.6654250621795654 -5.20071268081665 -

87.89374542236328 -2.784825325012207 -5.353398323059082 -4.9901580810546875 -5.817239761352539
87.82374572753906 -2.860379457473755 -5.474059104919434 -2.903717041015625 -5.403718948364258
88.13249969482422 -2.8344597816467285 -5.503992557525635 -2.33026123046875 -9.28814697265625
87.9887466430664 -2.858323335647583 -5.4693217277526855 -2.251861572265625 -5.003509521484375
88.2874984741211 -2.836028575897217 -5.441439151763916 -3.671076774597168 -5.889556884765625
88.04124450683594 -2.7648849487304688 -5.290419578552246 -2.531402587890625 -4.331451416015625
88.05874633789062 -2.8768224716186523 -5.523134708404541 -2.285799026489258 -3.9825592041015625
88.1312484741211 -2.8519651889801025 -5.487560749053955 -5.3453369140625 -12.92486572265625
88.125 -2.863798141479492 -5.542144775390625 -3.4252161979675293 -4.475826263427734
88.02249908447266 -2.904203414916992 -5.564382076263428 -2.814300537109375 -6.937934875488281
87.9000015258789 -2.857522487640381 -5.498369216918945 -1.26806640625 

88.37374877929688 -2.790005922317505 -5.344128131866455 -4.834716796875 -3.42205810546875
87.81124877929688 -2.786247491836548 -5.346039295196533 -2.1179351806640625 -4.583984375
88.19999694824219 -2.776484727859497 -5.30911922454834 -3.66632080078125 -5.633116722106934
88.18374633789062 -2.7802133560180664 -5.318333625793457 -1.4793319702148438 -2.5079593658447266
88.72249603271484 -2.766228675842285 -5.332205772399902 -2.4537200927734375 -9.097640991210938
88.22000122070312 -2.735391139984131 -5.228133201599121 -4.6805877685546875 -3.486845016479492
88.11499786376953 -2.7390103340148926 -5.2655863761901855 -2.984130859375 -4.347930908203125
88.14250183105469 -2.7168776988983154 -5.213233470916748 -3.51239013671875 -3.8686838150024414
88.37999725341797 -2.8202667236328125 -5.404062271118164 -3.05938720703125 -6.9502105712890625
88.33499908447266 -2.857093334197998 -5.467800617218018 -2.789031982421875 -0.7170867919921875
88.05374908447266 -2.8662109375 -5.487464904785156 -1.5843505859

87.97750091552734 -3.054455280303955 -5.808756351470947 -3.4937744140625 -9.045608520507812
87.92749786376953 -3.0757877826690674 -5.897982120513916 -2.915924072265625 -9.955665588378906
88.2074966430664 -3.0640172958374023 -5.807476043701172 -2.1287498474121094 -4.205322265625
88.50125122070312 -3.131010055541992 -5.85413932800293 -5.205078125 -5.204071044921875
88.16374969482422 -3.1286532878875732 -5.867167949676514 -2.900613784790039 -3.4429779052734375
87.86624908447266 -3.1038506031036377 -5.9307684898376465 -3.847320556640625 -11.585418701171875
88.25499725341797 -3.0886011123657227 -5.849029541015625 -3.27880859375 -8.24920654296875
88.04499816894531 -3.0585238933563232 -5.835195541381836 -2.1616058349609375 -5.9944000244140625
88.07624816894531 -3.0298943519592285 -5.811849594116211 -5.34185791015625 -8.114669799804688
88.03499603271484 -3.1078102588653564 -5.850144386291504 -5.374053955078125 -10.61810302734375
88.21749877929688 -3.0614640712738037 -5.763428211212158 -2.79426